# Distributed PyTorch Image Classification

**Learning Objectives** - By the end of this tutorial you should be able to use Azure Machine Learning (AzureML) to:
- quickly implement basic commands for data preparation
- assemble a pipeline with custom data preparation (python) scripts
- test and run a multi-node multi-gpu pytorch job
- register and deploy a pytorch model

**Requirements** - In order to benefit from this tutorial, you need:
- to have provisioned an AzureML workspace
- to have permissions to create simple SKUs in your resource group
- a python environment

**Motivations** - Let's consider the following scenario: we want to explore training different image classifiers on distinct kinds of problems, based on a large public dataset that is available at a given url. This ML pipeline will be future-looking, in particular we want:
- **genericity**: to be fairly independent from the data we're ingesting (so that we could switch to internal proprietary data in the future),
- **configurability**: to run different versions of that training with simple configuration changes,
- **scalability**: to iterate on the pipeline on small sample, then smoothly transition to running at scale.

### Connect to AzureML

Before we dive in the code, we'll need to create an instance of MLClient to connect to Azure ML. Please provide the references to your workspace below.

In [ ]:
# handle to the workspace
from azure.ml import MLClient

# authentication package
from azure.identity import InteractiveBrowserCredential

# get a handle to the workspace
ml_client = MLClient(
    InteractiveBrowserCredential(),
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group_name="<RESOURCE_GROUP>",
    workspace_name="<AML_WORKSPACE_NAME>",
)

# 1. Implement a reusable data preparation pipeline

To develop our data preparation pipeline, there are a couple constraints that we're setting for ourselves:
- we want to minimize the effort to ingest public data as it is used only as a learning opportunity,
- we do not want to manipulate large data locally (ex: download/upload that data could take multiple hours),

In this section, we'll achieve just that, by implementing the following:
- a data ingestion and processing pipeline with simple shell commands (wget, unzip) using minimal boilerplate code,

## 1.1. Unzip archives with a simple command (no code)

To train our classifier, we'll consume the [Common Objects in COntext (COCO) dataset](https://cocodataset.org/). If we were to use this locally, the sequence would be very basic: download 3 zip files, unzip each of them in a distinct folder for train/val/test, use python to extract annotations into a format we can use. We'll do just that, but in the cloud, without too much pain.

The Azure ML SDK provides `entities` to implement any step of a workflow. In the example below, we create a `CommandComponent` with just a shell command. We parameterize this command by using a string template syntax provided by the SDK:

> ```
> curl -o local_archive.zip ${{inputs.url}} && unzip local_archive.zip -d ${{outputs.extracted_data}}
> ```

Creating the component just consists in declaring the names of the inputs, outputs, and specifying an environment. For this simple job we'll use a curated environment from AzureML. After that, we'll be able to reuse that component multiple times in our pipeline design.

In [ ]:
from azure.ml.entities import CommandComponent, JobInput, JobOutput

download_unzip_component = CommandComponent(
    name="download_and_unzip",  # optional: this will show in the UI
    # this component has no code, just a simple unzip command
    command="curl -o local_archive.zip ${{inputs.url}} && unzip local_archive.zip -d ${{outputs.extracted_data}}",
    # I/O specifications, each using a specific key and type
    inputs={
        # 'url' is the key of this input string
        "url": {"type": "string"}
    },
    outputs={
        # 'extracted_data' will be the key to link this output to other steps in the pipeline
        "extracted_data": {"type": "path"}
    },
    # we're using a curated environment
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:9",
)

The component we just created can now be loaded as a component: a reusable step in a pipeline. 

In [ ]:
from azure.ml import dsl

# we'll package this unzip command as a component to use within a pipeline
download_unzip_component_func = dsl.load_component(component=download_unzip_component)

?download_unzip_component_func

## 1.2. Add a python script

Next step in our pipeline is to implement a simple script to extract the annotations and format them for us. We've written that script in this repository, and it can be loaded as a component from its yaml specification.

In [ ]:
from azure.ml import dsl

parse_annotations_func = dsl.load_component(
    yaml_file="./src/coco_extract_annotations/spec.yaml"
)

?parse_annotations_func

## 1.3. Write a reusable pipeline

We use the decorator `@dsl.pipeline` to construct an AzureML pipeline assembling the two components above.

In [ ]:
from azure.ml import dsl

# the dsl decorator tells the sdk that we are defining an AML pipeline
@dsl.pipeline(
    compute="cpu-cluster",
    description="e2e images preparation",
)
def coco_preparation_pipeline(
    train_archive_url,
    valid_archive_url,
    test_archive_url,
    annotations_archive_url,
    category_id,
    category_name,
):
    # 1st instance using the command component above on the training data
    train_unzip_step = download_unzip_component_func(url=train_archive_url)

    # 2nd instance for validation data
    valid_unzip_step = download_unzip_component_func(url=valid_archive_url)

    # 3rd instance for testing data
    test_unzip_step = download_unzip_component_func(url=test_archive_url)

    # 4th instance for the annotations
    annotations_unzip_step = download_unzip_component_func(url=annotations_archive_url)

    # add the annotations processing after the unzip command
    parse_annotations_step = parse_annotations_func(
        # here we consume the output of the unzip step
        annotations_dir=annotations_unzip_step.outputs.extracted_data,
        # parameters for this step are given as pipeline parameters
        # to allow for genericity (no hardcoded value)
        category_id=category_id,
        category_name=category_name,
    )

    # outputs of this pipeline are coded as a dictionary
    # keys can be used to assemble and link this pipeline with other pipelines
    return {
        "train_images": train_unzip_step.outputs.extracted_data,
        "valid_images": valid_unzip_step.outputs.extracted_data,
        "test_images": test_unzip_step.outputs.extracted_data,
        "train_annotations": parse_annotations_step.outputs.train_annotations,
        "valid_annotations": parse_annotations_step.outputs.valid_annotations,
    }

The pipeline we just created, decorated by `@dsl.pipeline` can also be called from python, as a sub-pipeline within another pipeline, creating more complex workflows (we'll see in next section).

In [ ]:
?coco_preparation_pipeline

## 1.4.. Run an instance of this pipeline in AzureML

When calling the pipeline function decorated with `@dsl.pipeline`, we will create an instance of this pipeline with the given arguments. In this scenario, we just provide the urls to the zip files we want to process, and the category of the objects we plan to train on.

In [ ]:
pipeline_instance = coco_preparation_pipeline(
    train_archive_url="http://images.cocodataset.org/zips/train2017.zip",
    valid_archive_url="http://images.cocodataset.org/zips/val2017.zip",
    test_archive_url="http://images.cocodataset.org/zips/test2017.zip",
    annotations_archive_url="http://images.cocodataset.org/annotations/annotations_trainval2017.zip",
    category_id=1,
    category_name="contains_person",
)

That instance can be submitted to AzureML and run there. Use the `MLClient` to create the job:

In [ ]:
# submit the pipeline job
returned_job = ml_client.jobs.create_or_update(
    pipeline_instance,
    # Project's name
    experiment_name="e2e_image_sample",
    # If there is no dependency, pipeline run will continue even after the failure of one component
    continue_run_on_step_failure=True,
)

# get a URL for the status of the job
print("The url to see your live job running is returned by the sdk:")
print(returned_job.services["Studio"].endpoint)

Considering the size of the dataset, this job will take a couple hours to complete. The validation and annotations dataset are smaller, and should take a couple minutes only to unzip. So while we wait for the training dataset (110k+ images) to finalize, you can already go into AzureML and register the outputs of the pipeline as datasets (see below).

<span style="color:red">IMPORTANT</span> - To move forward with the next section, we'll need you to:
- register the output of "valid_unzip_step" as dataset "coco_val2017"
- register the 2nd output of "Extract Coco Annotations" as dataset "coco_val2017_annotations"

![](media/image-prep-pipeline.png)

# 2. Training a distributed gpu job

Implementing a distributed pytorch training is complex. Of course in this tutorial we've written one for you, but the point is: it takes time, it takes several iterations, each requiring you to try your code locally, then in the cloud, then try it at scale, until satisfied and then run a full blown production model training. This trial/error process can be made easier if we can create reusable code we can iterate on quickly, and that can be configured to run from small to large scale.

So, to develop our training pipeline, we set a couple constraints for ourselves:
- we want to minimize the effort to iterate on the pipeline code when porting it in the cloud,
- we want to use the same code for small scale and large scale testing (so that we don't do not want to manipulate large data locally (ex: download/upload that data could take multiple hours),

## 2.1. Implement training and test on a sample

We've implemented a distributed pytorch training script that we can load as a component using its yaml specification, like we did for other components before.

Writing a pipeline to run it will be greatly simplified by the Azure ML SDK `@dsl.pipeline` decorator. For this, we've decided to parameterize this pipeline with relevant arguments (see below):

In [ ]:
from azure.ml import dsl

# load the component from its yaml specifications
training_func = dsl.load_component(yaml_file="./src/pytorch_dl_train/spec.yaml")

# the dsl decorator tells the sdk that we are defining an Azure ML pipeline
@dsl.pipeline(
    description="e2e images classification",  # TODO: document
)
def coco_model_training(
    train_images,  # dataset containing training images
    valid_images,  # dataset containing validation images
    train_annotations,  # annotations in CSV (see coco_extract_annotations/)
    valid_annotations,  # annotations in CSV (see coco_extract_annotations/)
    model_name,  # a name to register the model after training
    epochs,  # the number of epochs
    enabling_profiling,  # bonus: we've implemented pytorch profiling in our script
):
    # the training step is calling our training component with the right arguments
    training_step = training_func(
        # inputs
        train_images=train_images,
        valid_images=valid_images,
        train_annotations=train_annotations,
        valid_annotations=valid_annotations,
        # params (some hardcoded, some given by pipeline parameters)
        num_epochs=epochs,
        register_model_as=model_name,
        num_workers=-1,  # use all cpus (see train.py)
        # params (profiling)
        enabling_profiling=enabling_profiling,  # turns on profiler (see train.py)
        profiler_export_format="markdown",  # report in markdown format (see train.py)
    )
    # we set the name of the compute target for this training job
    training_step.compute = "gpu-cluster"

    # use process_count_per_instance to parallelize on multiple gpus
    training_step.distribution.process_count_per_instance = (
        1  # set to number of gpus on instance
    )

    # use instance_count to increase the number of nodes (machines)
    training_step.resources.instance_count = 1

    # outputs of this pipeline are coded as a dictionary
    # keys can be used to assemble and link this pipeline with other pipelines
    return {"model": training_step.outputs.trained_model}


# TODO: document
?coco_model_training

We can now test this code by running it on a smaller dataset in Azure ML. Here, we will use the validation set for training. Of course, the model will not be valid. But training will be short (8 mins on Standard_NC6 for 1 epoch) to allow us to iterate if needed.

In [ ]:
pipeline_instance = coco_model_training(
    # inputs: using validation set for training makes model invalid
    train_images=ml_client.datasets.get("coco_val2017", version=1),
    valid_images=ml_client.datasets.get("coco_val2017", version=1),
    train_annotations=ml_client.datasets.get("coco_val2017_annotations", version=2),
    valid_annotations=ml_client.datasets.get("coco_val2017_annotations", version=2),
    # training parameters surfaced in the pipeline definition
    epochs=1,  # 1 epoch only for testing, model isn't valid anyway
    model_name="coco_model_person_dev",
    enabling_profiling=True,  # turns on profiler (see train.py)
)

Once we create that pipeline instance, we submit it through `MLClient`.

In [ ]:
# submit the pipeline job
returned_job = ml_client.jobs.create_or_update(
    pipeline_instance,
    # Project's name
    experiment_name="e2e_image_sample",
    # If there is no dependency, pipeline run will continue even after the failure of one component
    continue_run_on_step_failure=True,
)

# get a URL for the status of the job
print("The url to see your live job running is returned by the sdk:")
print(returned_job.services["Studio"].endpoint)

You can iterate on this design as much as you'd like, updating the local code of the component and re-submit the pipeline. You would have to go back to the beginning of this section (from the `dsl.load_component()` call) in order to refresh the pipeline definition with a new version of the code.

## 2.2. Use MLFlow to report metrics, logs, artifacts

Azure ML natively integrated MLFlow so that if your code already supports MLFlow logging, you will not have to modify it to report your metrics within Azure ML.

The component above is using MLFlow internally to report relevant metrics, logs and artifacts. Look for `mlflow` calls within the script `train.py`.

To access this data, open the pipeline in the Azure ML Studio.

![](media/pytorch_train_pipeline.png)

Click on the component to display the Details panel. Choose **Metrics** to check the values reported using [`mlflow.log_metric()`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_metric).

![](media/pytorch_train_metrics.png)

Your component can also report files using [`mlflow.log_artifact()`](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_artifact). NAvigate in the **Outputs+Logs** panel to find the artifacts. In our implementation, we use it to log a markdown file containing the output of [PyTorch profiler](https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html#using-profiler-to-analyze-execution-time).

![](media/pytorch_train_profile.png)

## 2.3. Reuse the same pipeline on full scale data

Once the pipeline is satisfying, running it full scale is actually just adjusting the inputs and parameters. We're creating a new instance below where we use training images as training set (as we should).

In [ ]:
pipeline_instance = coco_model_training(
    train_images=ml_client.datasets.get("coco_train2017", version=1),
    valid_images=ml_client.datasets.get("coco_val2017", version=1),
    train_annotations=ml_client.datasets.get("coco_train2017_annotations", version=2),
    valid_annotations=ml_client.datasets.get("coco_val2017_annotations", version=2),
    epochs=10,
    model_name="coco_model_person_full",
    profile=False,  # turns off profiler (see train.py)
)

In [ ]:
# submit the pipeline job
returned_job = ml_client.jobs.create_or_update(
    pipeline_instance,
    # Project's name
    experiment_name="e2e_image_sample",
    # If there is no dependency, pipeline run will continue even after the failure of one component
    continue_run_on_step_failure=True,
)

# get a URL for the status of the job
print("The url to see your live job running is returned by the sdk:")
print(returned_job.services["Studio"].endpoint)

# 3. Deploy the Model as an Online Endpoint

As mentioned above, the model will be registered within the training pipeline at the end of the epochs. Next step is to deploy this registered model in an online endpoint for inferencing.

An endpoint in AzureML is creating the URL behind which our model will be hosted. Creating that endpoint does not deploy the model. Instead, you can deploy several models behind this endpoint and manage traffic coming through each of them.

In this tutorial, we'll just deploy one version of the model at full traffic.

## 3.1. Create an endpoint

We'll use the `MLClient` to create an endpoint with a unique name.

In [ ]:
from azure.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)
import uuid

# Generate a unique name for the endpoint
# NOTE: endpoint names have to be unique at the Azure region level
online_endpoint_name = "coco-person-" + str(uuid.uuid4())[:8]


# Create an online endpoint object
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="online endpoint for pytorch image classifier",
    auth_mode="key",
    tags={"model": "vision"},
)

# Use MLClient to create the endpoint
endpoint = ml_client.begin_create_or_update(endpoint)

# Report and track the status
print(f'Endpoint "{endpoint.name}" provisioning state: {endpoint.provisioning_state}')

## 3.2. Deploy the model to the endpoint

And in a second step, deploy the registered model in that endpoint. This operation could take approx 20 minutes to complete.

In [ ]:
from azure.ml.entities import (
    ManagedOnlineDeployment,
    Environment,
)

# name of the model we want to deploy
registered_model_name = "coco_model_person_dev"

# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)

# create an online deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint.name,
    model=f"{registered_model_name}:{latest_model_version}",
    environment="AzureML-pytorch-1.10-ubuntu18.04-py38-cuda11-gpu:11",
    code_local_path="./src/deployment/",
    scoring_script="score.py",
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

ml_client.begin_create_or_update(blue_deployment)

## 3.3. Test the endpoint with REST calls

To check our model deployment, we will try it out on two images sent to the endpoint using simple requests.

We'll use the two images below from their public urls, download them and encode them to build a json payload.

| `non_person_image.jpg` |  `person.jpg` |
| :-- | :-- |
| <img src="http://images.cocodataset.org/val2017/000000567197.jpg" alt="a black and white picture of a warehouse" width="30%"> | <img src="http://images.cocodataset.org/val2017/000000274219.jpg" alt="a black and white picture of a man" width="30%"> |

The python code below will then call the endpoint URI and show you the returned response from the model we just deployed.

In [ ]:
import requests
import base64
import traceback

# download each image locally
non_person_image = requests.get(
    "http://images.cocodataset.org/val2017/000000567197.jpg"
).content
with open("./non_person_image.jpg", "wb") as handler:
    handler.write(non_person_image)

person_image = requests.get(
    "http://images.cocodataset.org/val2017/000000274219.jpg"
).content
with open("./person_image.jpg", "wb") as handler:
    handler.write(person_image)

# from local image, create encoded versions to send as requests
with open("./non_person_image.jpg", mode="rb") as ifile:
    non_person_image_encoded = base64.b64encode(ifile.read()).decode("ascii")

with open("./person_image.jpg", mode="rb") as ifile:
    person_image_encoded = base64.b64encode(ifile.read()).decode("ascii")

request_payload = {
    "rows": [{"image": non_person_image_encoded}, {"image": person_image_encoded}]
}

try:
    print(f"URI: {endpoint.scoring_uri}")
    r = requests.post(endpoint.scoring_uri, json=request_payload)

    if r.status_code == 200:
        print(f"Endpoint returned: {r.json()}")
    else:
        print(f"Endpoint returned: {r.text}")
except:
    print(traceback.format_exc())